<a href="https://colab.research.google.com/github/Aryan011001/3D_Portfolio/blob/main/face_morphing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [9]:
images_path ="/content/drive/MyDrive/Colab Notebooks/FACE_MORPHING/ArshJeetSent/rgb_data.npy"
labels_path="/content/drive/MyDrive/Colab Notebooks/FACE_MORPHING/ArshJeetSent/mask_data.npy"

In [10]:
from sklearn.model_selection import train_test_split
import cv2
from os import listdir
from os.path import isdir, join
import numpy as np

# Loading the preprocessed data using numpy

In [11]:
rgb_images = np.load(images_path)
resized_masks_array = np.load(labels_path)

# Splitting the dataset


In [12]:
def split_data(rgb_images, resized_masks_array, validation_split):
  # Split data into training and validation sets
  num_samples = len(rgb_images)
  num_train_samples = int((1 - validation_split) * num_samples)

  # Separate training and validation data
  train_rgb_images = rgb_images[:num_train_samples]
  train_masks = resized_masks_array[:num_train_samples]

  val_rgb_images = rgb_images[num_train_samples:]
  val_masks = resized_masks_array[num_train_samples:]

  # return (train_rgb_images, train_masks), (val_rgb_images, val_masks)

  train_masks = np.expand_dims(train_masks, axis=-1)  # Add channel dimension at the end
  val_masks = np.expand_dims(val_masks, axis=-1)

  return (train_rgb_images, train_masks), (val_rgb_images, val_masks)

In [13]:
(train_rgb_images, train_masks), (test_rgb_images, test_masks) = split_data(rgb_images, resized_masks_array, validation_split=0.1)
(train_rgb_images, train_masks), (val_rgb_images, val_masks) = split_data(train_rgb_images, train_masks, validation_split=0.1)

# Create datasets with masks

In [14]:
import tensorflow as tf

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_rgb_images, train_masks)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_rgb_images, val_masks)).batch(1)
test_dataset = tf.data.Dataset.from_tensor_slices((test_rgb_images, test_masks)).batch(1)

# Shapes of images

In [16]:
print(train_rgb_images.shape)  # Print the shape of train_rgb_images
print(train_masks.shape)       # Print the shape of train_masks

(1932, 64, 64, 3)
(1932, 64, 64, 1, 1)


In [17]:
for images, masks in train_dataset.take(1):  # Took 1 batch from the training data
    print(images.shape)  # input images shapes
    print(masks.shape)   # shape of the masks

(32, 64, 64, 3)
(32, 64, 64, 1, 1)


# for saving each epoch
creating callbacks and checkpoints

In [2]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/FACE_MORPHING/training_checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# CNN Architecture and Compiling

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid'),
    # tf.keras.layers.Dense(14, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')  # Output shape matches the shape of the labels
# ])

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 1)         65        
                                                                 
Total params: 19457 (76.00 KB)
Trainable params: 19457 (

In [5]:
model.add(Flatten())

In [6]:
if os.path.exists(checkpoint_dir):
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint is not None:
        print("Loading weights from", latest_checkpoint)
        model.load_weights(latest_checkpoint)

# Training the model

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
 2/61 [..............................] - ETA: 31:35 - loss: 0.5596 - accuracy: 0.0138    

# Evaluating the model

In [19]:
# test_loss, test_accuracy = model.evaluate(test_dataset)
# print(f'Test Loss: {test_loss}')
# print(f'Test Accuracy: {test_accuracy}')

# Evaluate the model
evaluation_results = model.evaluate(test_dataset)

# Check if evaluation results are valid
if evaluation_results is not None and len(evaluation_results) == 2:
    test_loss, test_accuracy = evaluation_results
    print(f'Test Loss: {test_loss}')
    print(f'Test Accuracy: {test_accuracy}')
else:
    print("Error: Evaluation results are not valid or None")


239/239 [==============================] - 2s 6ms/step - loss: 0.6776 - accuracy: 0.0000e+00
Test Loss: 0.6775946617126465
Test Accuracy: 0.0
